## standard imports

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import MDAnalysis as mda

from modules.general import flatten
from modules.traj import System, TrajectorySlice
from modules.constants import PATH, EXPERIMENTS

sns.set(
    context="notebook", palette="muted", style="ticks", rc={"figure.figsize": (9, 6)}
)

<IPython.core.display.Javascript object>

## visualize trajectory

In [9]:
import nglview as nv

syst = "dops_chol50"
u = mda.Universe(
    str(PATH / syst / "md" / "md.tpr"),
    str(PATH / syst / "md" / "pbcmol_199-200-10.xtc"),
)
lips = u.select_atoms("resname DOPS or resname CHL")
view = nv.show_mdanalysis(lips)
view.background = "black"
view

NGLWidget(background='black', max_frame=100)

<IPython.core.display.Javascript object>

In [17]:
view.clear_representations()
view.add_ball_and_stick("CHL", color="cyan")
view.add_ball_and_stick("DOPS")
view.player.parameters = dict(delay=0.4)

<IPython.core.display.Javascript object>

## caluclate velocities using .xtc

In [ ]:
syst = "popc_chol10"
print(syst)
print("importing universe...")
u = mda.Universe(
    str(PATH / syst / "md" / "md.tpr"),
    str(PATH / syst / "md" / "md.xtc"),
)

In [ ]:
print("collecting positions...")
at_pos = {}
for ts in u.trajectory[(199000 - 1) : 199001]:
    print(ts.time)
    at_pos[u.trajectory.frame] = u.atoms.positions

print("calculating velocities...")
velocities = (at_pos[199000] - at_pos[199000 - 1]) / u.trajectory.dt

u.trajectory[199000]
u.trajectory.ts.has_velocities = True
u.atoms.positions = at_pos[199000]
u.atoms.velocities = velocities

In [ ]:
print("saving md_19000.trr...")
with mda.Writer(str(PATH / syst / "md" / "md_199000.trr"), u.atoms.n_atoms) as writer:
    writer.write(u.atoms)
print("done.")

### run grompp

In [5]:
import subprocess

syst = "popc_chol30"

cmd = " ".join(
    [
        ". /usr/local/gromacs-2020.4/bin/GMXRC && ",
        f"gmx grompp -f {str(PATH / syst)}/md_last_ns.mdp",
        f"-c {str(PATH / syst)}/pr4/pr4.gro",
        f"-r {str(PATH / syst)}/pr4/pr4.gro",
        f"-p {str(PATH / syst)}/indata/system.top",
        f"-n {str(PATH / syst)}/indata/grps.ndx",
        f"-t {str(PATH / syst)}/md/md_199000.trr",
        # f'-e {str(PATH / syst)}/md/md.edr',
        f"-o {str(PATH / syst)}/md/last_ns.tpr -v",
    ]
)
print(cmd)

. /usr/local/gromacs-2020.4/bin/GMXRC &&  gmx grompp -f /home/klim/Documents/chol_impact/popc_chol30/md_last_ns.mdp -c /home/klim/Documents/chol_impact/popc_chol30/pr4/pr4.gro -r /home/klim/Documents/chol_impact/popc_chol30/pr4/pr4.gro -p /home/klim/Documents/chol_impact/popc_chol30/indata/system.top -n /home/klim/Documents/chol_impact/popc_chol30/indata/grps.ndx -t /home/klim/Documents/chol_impact/popc_chol30/md/md_199000.trr -o /home/klim/Documents/chol_impact/popc_chol30/md/last_ns.tpr -v


<IPython.core.display.Javascript object>

In [6]:
subprocess.run(cmd, shell=True, check=True)

                      :-) GROMACS - gmx grompp, 2020.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

processing topology...
turning H bonds into constraints...
turning H bonds into constraints...
turning H bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 310 K
Calculated rlist for 1x1 atom pair-list as 1.524 nm, buffer size 0.024 nm
Set rlist, assuming 4x4 atom pair-list, to 1.500 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 60x60x108, spacing 0.115 0.115 0.119
This run will generate roughly 357 Mb of data


double-checking input for internal consistency...
renumbering atomtypes...
converting bonded parameters...
initialising group options...
processing index file...
Making dummy/rest group for Acceleration containing 62464 elements
Making dummy/rest group for Freeze containing 62464 elements
Number of degrees of freedom in T-Coupling group LIPIDS is 54493.00
Number of degrees of freedom in T-Coupling group SOL is 79245.00
Making dummy/rest group for User1 containing 62464 elements
Making dummy/rest group for User2 containing 62464 elements
Making dummy/rest group for Compressed X containing 62464 elements
Making dummy/rest group for Or. Res. Fit containing 62464 elements
Making dummy/rest group for QMMM containing 62464 elements
T-Coupling       has 2 element(s): LIPIDS SOL
Energy Mon.      has 1 element(s): SYS_ALL
Acceleration     has 1 element(s): rest
Freeze           has 1 element(s): rest
User1            has 1 element(s): rest
User2            has 1 element(s): rest
VCM            

CompletedProcess(args='. /usr/local/gromacs-2020.4/bin/GMXRC &&  gmx grompp -f /home/klim/Documents/chol_impact/popc_chol30/md_last_ns.mdp -c /home/klim/Documents/chol_impact/popc_chol30/pr4/pr4.gro -r /home/klim/Documents/chol_impact/popc_chol30/pr4/pr4.gro -p /home/klim/Documents/chol_impact/popc_chol30/indata/system.top -n /home/klim/Documents/chol_impact/popc_chol30/indata/grps.ndx -t /home/klim/Documents/chol_impact/popc_chol30/md/md_199000.trr -o /home/klim/Documents/chol_impact/popc_chol30/md/last_ns.tpr -v', returncode=0)

<IPython.core.display.Javascript object>

## check last_ns.gro in all systems

In [6]:
systems = flatten(
    [
        (i, i + "_chol10", i + "_chol30", i + "_chol50")
        for i in flatten(EXPERIMENTS.values())
    ]
)
not_yet = []
for syst in systems:
    if not (PATH / syst / "md" / "last_ns.xtc").is_file():
        not_yet.append(syst)
print(not_yet)

['popc', 'popc_chol30', 'dops', 'dops_chol30']


<IPython.core.display.Javascript object>